In [3]:
## 라이브러리 임포트
import requests
import pprint
import json
import pandas as pd
import os
from datetime import date, timedelta, datetime
from dotenv import load_dotenv
load_dotenv()
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

### 자전거 대여 기록

In [ ]:
class SeoulBikeAPI:
       
    def __init__(self, api_key=None):
        # API 키 설정 (환경 변수 또는 직접 입력)
        self.api_key = api_key or os.getenv('seoul_api_key')
        if not self.api_key:
            raise ValueError("API Key is required. Set it as an environment variable or pass it explicitly.")
    
    def generate_date_range(self, start, end):
        """날짜 범위를 생성하는 제너레이터"""
        current_date = start
        while current_date <= end:
            yield current_date
            current_date += timedelta(days=1)
    
    def fetch_data(self, start=date(2024, 6, 1), end=date(2024, 12, 31)):
        """지정된 날짜 범위에 대해 데이터를 가져옵니다."""
        if start > end:
            raise ValueError("Start date must be earlier than or equal to the end date.")
        
        all_data = []  # 모든 날짜의 데이터를 저장할 리스트
        
        for single_date in self.generate_date_range(start, end):
            formatted_date = single_date.strftime("%Y-%m-%d")
            
            for hour in range(0, 24):  # 시간별 데이터 호출
                for start_idx in range(1, 20001, 1000):  # 1~1000, 1001~2000 ... 호출
                    end_idx = start_idx + 999
                    
                    url = f'http://openapi.seoul.go.kr:8088/{self.api_key}/json/tbCycleRentData/{start_idx}/{end_idx}/{formatted_date}/{hour}'
                    
                    try:
                        response = requests.get(url)
                        response.raise_for_status()  # HTTP 에러 발생 시 예외 처리
                        data = response.json()
                        
                        # JSON 데이터에서 'rentData' -> 'row' 항목 추출
                        if 'rentData' in data and 'row' in data['rentData']:
                            all_data.extend(data['rentData']['row'])  # 리스트에 추가
                        else:
                            print(f"No data found for {formatted_date} hour {hour}, range {start_idx}-{end_idx}")
                            break  # 데이터가 없으면 해당 시간대 종료
                    
                    except requests.exceptions.RequestException as e:
                        print(f"Error fetching data for {formatted_date} hour {hour}, range {start_idx}-{end_idx}: {e}")
                        break  # 오류 발생 시 다음 시간대로 넘어감
        
        return all_data
    
    def fetch_data_as_dataframe(self, start=date(2024, 6, 1), end=date(2024, 12, 31)):
        """지정된 날짜 범위에 대해 데이터를 DataFrame 형식으로 반환합니다."""
        all_data = self.fetch_data(start=start, end=end)
        print("모든 데이터 호출 완료!")
        print("데이터 프레임 변환 시작!")
        # 리스트를 DataFrame으로 변환
        if all_data:
            df = pd.DataFrame(all_data)
            print("데이터 프레임 변환 끝!")
            return df
        else:
            print("No data available to create a DataFrame.")
            return pd.DataFrame()  # 빈 DataFrame 반환


In [8]:
# 데이터프레임 생성

api_key = os.getenv('seoul_api_key')

# 객체 생성
bike_api = SeoulBikeAPI(api_key)

for month in range(7, 13):
    # 원하는 날짜 범위를 설정
    start_date = date(2024, month, 1)
    end_date = (start_date.replace(day=28) + timedelta(days=4)).replace(day=1) - timedelta(days=1)  # 해당 월의 마지막 날 계산

    # 데이터를 DataFrame으로 가져오기
    df = bike_api.fetch_data_as_dataframe(start=start_date, end=end_date)

    # CSV 파일로 저장
    df.to_csv(f'./raw_data/rental_history_raw_{start_date}-{end_date}.csv', sep=',', encoding='utf-8', index=False)

No data found for 2024-07-01 hour 0, range 4001-5000
No data found for 2024-07-01 hour 1, range 2001-3000
No data found for 2024-07-01 hour 2, range 2001-3000
No data found for 2024-07-01 hour 3, range 1001-2000
No data found for 2024-07-01 hour 4, range 1001-2000
No data found for 2024-07-01 hour 5, range 3001-4000
No data found for 2024-07-01 hour 6, range 6001-7000
No data found for 2024-07-01 hour 7, range 13001-14000
No data found for 2024-07-01 hour 8, range 16001-17000
No data found for 2024-07-01 hour 9, range 8001-9000
No data found for 2024-07-01 hour 10, range 6001-7000
No data found for 2024-07-01 hour 11, range 7001-8000
No data found for 2024-07-01 hour 12, range 7001-8000
No data found for 2024-07-01 hour 13, range 6001-7000
No data found for 2024-07-01 hour 14, range 6001-7000
No data found for 2024-07-01 hour 15, range 7001-8000
No data found for 2024-07-01 hour 16, range 10001-11000
No data found for 2024-07-01 hour 17, range 16001-17000
No data found for 2024-07-01 h

### 자전거 대여소 위도, 경도

In [4]:
# 자전거 위도, 경도 데이터
class SeoulBikeInfo:
    def __init__(self, api_key=None):
        """
        API 키를 초기화합니다.
        :param api_key: 서울시 공공데이터 API 키 (환경 변수 또는 직접 입력)
        """
        self.api_key = api_key or os.getenv('seoul_api_key')
        if not self.api_key:
            raise ValueError("API Key가 필요합니다. 환경 변수로 설정하거나 직접 전달하세요.")
    
    def fetch_bike_list(self, start=1, end=1000):
        """
        서울시 공공자전거 대여소 정보를 가져옵니다.
        :param start: 데이터 시작 위치 (기본값: 1)
        :param end: 데이터 종료 위치 (기본값: 1000)
        :return: JSON 데이터
        """
        url = f'http://openapi.seoul.go.kr:8088/{self.api_key}/json/bikeList/{start}/{end}/'
        
        try:
            response = requests.get(url)
            response.raise_for_status()  # HTTP 에러 발생 시 예외 처리
            data = response.json()
            
            # rentBikeStatus -> row 데이터 반환
            if 'rentBikeStatus' in data and 'row' in data['rentBikeStatus']:
                return data['rentBikeStatus']['row']
            else:
                raise ValueError("API 응답에 'rentBikeStatus' 또는 'row' 데이터가 없습니다.")
        
        except requests.exceptions.RequestException as e:
            print(f"데이터를 가져오는 중 오류가 발생했습니다: {e}")
            return []
    
    def get_bike_data_as_dataframe(self, start=1, end=1000):
        """
        대여소 데이터를 DataFrame 형식으로 반환합니다.
        :param start: 데이터 시작 위치 (기본값: 1)
        :param end: 데이터 종료 위치 (기본값: 1000)
        :return: pandas DataFrame
        """
        bike_data = self.fetch_bike_list(start=start, end=end)
        
        if bike_data:
            # JSON 데이터를 DataFrame으로 변환
            df = pd.DataFrame(bike_data)
            return df
        else:
            print("가져온 데이터가 없습니다.")
            return pd.DataFrame()  # 빈 DataFrame 반환


In [5]:
# # API 키 설정 (환경 변수 또는 직접 입력)
# api_key = os.getenv('seoul_api_key')

# # 객체 생성
# bike_api = SeoulBikeInfo(api_key)

# # 대여소 정보를 DataFrame으로 변환
# bike_df = bike_api.get_bike_data_as_dataframe(start=1, end=1000)
# bike_df2 = bike_api.get_bike_data_as_dataframe(start=1001, end=2000)
# bike_df3 = bike_api.get_bike_data_as_dataframe(start=2001, end=3000)

# bike_info = pd.concat([bike_df, bike_df2, bike_df3], ignore_index=True)

# # CSV 파일로 저장
# bike_info.to_csv("seoul_bike_info.csv", sep = ',', encoding='utf-8', index=False)


### 자전거 대여소별 대여 건수 데이터

In [6]:
# 자전거 대여 건수 데이터 호출
class SeoulBikeUsage:
    def __init__(self, api_key=None):
        """
        API 키를 초기화합니다.
        :param api_key: 서울시 공공데이터 API 키 (환경 변수 또는 직접 입력)
        """
        self.api_key = api_key or os.getenv('seoul_api_key')
        if not self.api_key:
            raise ValueError("API Key가 필요합니다. 환경 변수로 설정하거나 직접 전달하세요.")
    
    def generate_date_range(self, start_date, end_date):
        """
        시작 날짜와 종료 날짜 사이의 날짜 범위를 생성합니다.
        :param start_date: 시작 날짜 (문자열 형식: 'YYYY-MM-DD')
        :param end_date: 종료 날짜 (문자열 형식: 'YYYY-MM-DD')
        :return: 날짜 문자열의 제너레이터
        """
        current_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
        
        while current_date <= end_date:
            yield current_date.strftime("%Y-%m-%d")
            current_date += timedelta(days=1)
    
    def fetch_data(self, start_date="2024-01-01", end_date="2024-12-31", row_start=1, row_end=1000):
        """
        지정된 날짜 범위의 데이터를 API로 호출하여 수집합니다.
        :param start_date: 데이터 수집 시작 날짜 (기본값: '2024-01-01')
        :param end_date: 데이터 수집 종료 날짜 (기본값: '2024-12-31')
        :param row_start: 데이터 시작 위치 (기본값: 1)
        :param row_end: 데이터 종료 위치 (기본값: 1000)
        :return: JSON 데이터 리스트
        """
        all_data = []  # 모든 데이터를 저장할 리스트
        
        for single_date in self.generate_date_range(start_date, end_date):
            url = f'http://openapi.seoul.go.kr:8088/{self.api_key}/json/tbCycleUseStatus/{row_start}/{row_end}/{single_date}'
            
            try:
                response = requests.get(url)
                response.raise_for_status()  # HTTP 에러 발생 시 예외 처리
                data = response.json()
                
                # useStatus -> row 데이터 추가
                if 'useStatus' in data and 'row' in data['useStatus']:
                    all_data.extend(data['useStatus']['row'])
                else:
                    print(f"{single_date}에 데이터가 없습니다.")
            
            except requests.exceptions.RequestException as e:
                print(f"{single_date} 데이터를 가져오는 중 오류가 발생했습니다: {e}")
        
        return all_data
    
    def fetch_data_as_dataframe(self, start_date="2024-01-01", end_date="2024-12-31", row_start=1, row_end=1000):
        """
        지정된 날짜 범위의 데이터를 DataFrame 형식으로 반환합니다.
        :param start_date: 데이터 수집 시작 날짜 (기본값: '2024-01-01')
        :param end_date: 데이터 수집 종료 날짜 (기본값: '2024-12-31')
        :param row_start: 데이터 시작 위치 (기본값: 1)
        :param row_end: 데이터 종료 위치 (기본값: 1000)
        :return: pandas DataFrame
        """
        all_data = self.fetch_data(start_date=start_date, end_date=end_date, row_start=row_start, row_end=row_end)
        
        if all_data:
            # JSON 데이터를 DataFrame으로 변환
            df = pd.DataFrame(all_data)
            return df
        else:
            print("가져온 데이터가 없습니다.")
            return pd.DataFrame()  # 빈 DataFrame 반환


In [7]:
# # API 키 설정 (환경 변수 또는 직접 입력)
# api_key = os.getenv('seoul_api_key')

# # 객체 생성
# bike_usage_api = SeoulBikeUsage(api_key)

# # 2024년 1월 1일부터 2024년 12월 31일까지 데이터를 DataFrame으로 가져오기
# bike_usage_df = bike_usage_api.fetch_data_as_dataframe(start_date="2024-01-01", end_date="2024-12-31")
# bike_usage_df2 = bike_usage_api.fetch_data_as_dataframe(start_date="2024-01-01", end_date="2024-12-31", row_start=1001, row_end=2000)
# bike_usage_df3 = bike_usage_api.fetch_data_as_dataframe(start_date="2024-01-01", end_date="2024-12-31", row_start=2001, row_end=3000)

# bike_usage_df = pd.concat([bike_usage_df, bike_usage_df2, bike_usage_df3], ignore_index=True)

# # CSV 파일로 저장
# bike_usage_df.to_csv("seoul_bike_usage.csv", index=False)


### 대여소 주변 핫플레이스 정보

In [ ]:
# 대여소 주변 관광지 정보

class SeoulBikeHotPlaces:
    def __init__(self, api_key=None):
        """
        Google Places API 키를 초기화합니다.
        :param api_key: Google Places API 키 (환경 변수 또는 직접 입력)
        """
        self.api_key = api_key or os.getenv('gmaps_api_key')
        if not self.api_key:
            raise ValueError("Google Places API Key가 필요합니다. 환경 변수로 설정하거나 직접 전달하세요.")
    
    def get_hot_places(self, latitude, longitude, radius=500, place_type="tourist_attraction", max_results=10):
        """
        특정 위치 주변의 핫플레이스를 검색합니다.
        :param latitude: 검색 중심점의 위도
        :param longitude: 검색 중심점의 경도
        :param radius: 검색 반경 (미터 단위, 기본값: 500m)
        :param place_type: 검색할 장소 유형 (예: restaurant, cafe, park 등)
        :param max_results: 반환할 최대 결과 수 (기본값: 10)
        :return: 핫플레이스 정보 리스트
        """
        base_url = "https://places.googleapis.com/v1/places:searchNearby"
        
        # 요청 본문 (JSON 형식)
        request_body = {
            "includedTypes": [place_type],
            "maxResultCount": max_results,
            "locationRestriction": {
                "circle": {
                    "center": {
                        "latitude": latitude,
                        "longitude": longitude
                    },
                    "radius": radius
                }
            }
        }
        
        # 요청 헤더
        headers = {
            "Content-Type": "application/json",
            "X-Goog-Api-Key": self.api_key,
            "X-Goog-FieldMask": "places.displayName,places.location,places.rating,places.userRatingCount"
        }
        
        try:
            # POST 요청 전송
            response = requests.post(base_url, json=request_body, headers=headers)
            response.raise_for_status()  # HTTP 에러 발생 시 예외 처리
            
            # 응답 데이터 파싱
            data = response.json()
            
            # 결과에서 필요한 정보 추출
            places = []
            for place in data.get("places", []):
                places.append({
                    "name": place.get("displayName", {}).get("text", "Unknown"),
                    "latitude": place["location"]["latitude"],
                    "longitude": place["location"]["longitude"],
                    "rating": place.get("rating"),
                    "user_ratings_total": place.get("userRatingCount")
                })
            
            return places
        
        except requests.exceptions.RequestException as e:
            print(f"데이터를 가져오는 중 오류가 발생했습니다: {e}")
            return []
    
    def get_places_for_stations(self, bike_stations_df, radius=500, place_type="tourist_attraction", max_results=10):
        """
        자전거 대여소 데이터프레임을 기반으로 각 대여소 주변 핫플레이스를 검색합니다.
        :param bike_stations_df: 자전거 대여소 정보가 포함된 DataFrame (latitude와 longitude 열 포함)
        :param radius: 검색 반경 (미터 단위, 기본값: 500m)
        :param place_type: 검색할 장소 유형 (예: restaurant, cafe, park 등)
        :param max_results: 반환할 최대 결과 수 (기본값: 10)
        :return: 대여소별 핫플레이스 정보를 포함한 DataFrame
        """
        all_places = []
        
        for _, row in bike_stations_df.iterrows():
            station_id = row["stationId"]
            station_name = row["stationName"]
            latitude = row["stationLatitude"]
            longitude = row["stationLongitude"]
            
            print(f"대여소 '{station_name}' 주변 핫플레이스 검색 중...")
            
            # 특정 대여소 주변 핫플레이스 검색
            places = self.get_hot_places(latitude, longitude, radius=radius, place_type=place_type, max_results=max_results)
            
            # 각 장소에 대여소 ID와 이름 추가
            for place in places:
                place["stationId"] = station_id
                place["stationName"] = station_name
            
            all_places.extend(places)
        
        # 결과를 DataFrame으로 변환
        return pd.DataFrame(all_places)


In [ ]:
# # Google Places API 키 설정 (환경 변수 또는 직접 입력)
# api_key = os.getenv('gmaps_api_key')

# # 객체 생성
# hot_places_api = SeoulBikeHotPlaces(api_key)

# # 자전거 대여소 데이터
# bike_stations_df = bike_info[['stationName', 'stationLatitude', 'stationLongitude', 'stationId']]
# bike_stations_df.drop_duplicates(inplace=True)

# # 각 대여소 주변 500m 반경의 관광지(Tourist Attraction) 정보 가져오기
# hot_places_df = hot_places_api.get_places_for_stations(bike_stations_df, radius=500, place_type="cafe")

# # CSV 파일로 저장
# hot_places_df.to_csv("bike_station_hot_places.csv", index=False, sep=',', encoding='utf-8')


### 날씨 데이터 API 호출

In [ ]:
# 기상청 API 허브 데이터 호출
class KMADataAPI:
    def __init__(self, api_key):
        """
        기상청 API 클래스 초기화
        :param api_key: 기상청 API 인증키
        """
        self.api_key = api_key
        self.base_url = "https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php"
        self.raw_data = None
        self.columns = ["TM", "STN", "TA", "HM", "RN", "WS", "SD_TOT", "RN_JUN"]
        self.data = []

    def fetch_data(self, tm, stn):
        """
        기상청 API에서 데이터를 가져옵니다.
        :param tm: 관측 시각 (예: '202401010000')
        :param stn: 관측소 번호 (예: '108'은 서울)
        """
        params = {
            "tm": tm,
            "stn": stn,
            "help": 0,  # 도움말 제거
            "authKey": self.api_key,
        }
        
        # API 호출
        response = requests.get(self.base_url, params=params)
        
        if response.status_code == 200:
            self.raw_data = response.text
            print(f"API 호출 성공! 시각: {tm}")
        else:
            raise Exception(f"API 호출 실패! 상태 코드: {response.status_code}, 시각: {tm}")

    def parse_data(self):
        """
        텍스트 데이터에서 실제 관측 데이터를 추출합니다.
        """
        if not self.raw_data:
            raise ValueError("데이터가 없습니다. 먼저 fetch_data를 호출하세요.")
        
        lines = self.raw_data.split("\n")
        
        # 데이터 행 추출 (숫자로 시작하는 행)
        data_lines = [line for line in lines if re.match(r"^\d", line)]
        
        for line in data_lines:
            # 필요한 열만 고정된 폭으로 분리
            row = [
                line[0:10].strip(),  # TM (관측 시각)
                line[12:16].strip(), # STN (관측소 ID)
                line[63:68].strip(), # TA (기온)
                line[75:80].strip(), # HM (습도)
                line[87:93].strip(), # RN (강수량)
                line[21:25].strip(), # WS (풍속)
                line[131:136].strip(), # SD_TOT (적설량)
                line[104:109].strip()  # RN_JUN (일강수량)
            ]
            self.data.append(row)

    def to_dataframe(self):
        """
        파싱한 데이터를 판다스 데이터프레임으로 변환합니다.
        """
        if not self.data:
            raise ValueError("Data가 없습니다. parse_data를 먼저 실행하세요.")
        
        df = pd.DataFrame(self.data, columns=self.columns)
        return df

def fetch_all_data(api_key, start_time, end_time, stn):
    """
    주어진 시간 범위 내에서 데이터를 반복적으로 호출하여 병합합니다.
    :param api_key: 기상청 API 인증키
    :param start_time: 시작 시각 (datetime 객체)
    :param end_time: 종료 시각 (datetime 객체)
    :param stn: 관측소 번호
    :return: 병합된 데이터프레임
    """
    kma_api = KMADataAPI(api_key)
    current_time = start_time
    all_data = []

    while current_time <= end_time:
        tm_str = current_time.strftime("%Y%m%d%H%M")  # 'YYYYMMDDHHMM' 형식으로 변환
        
        try:
            kma_api.fetch_data(tm=tm_str, stn=stn)  # API 호출
            kma_api.parse_data()                   # 데이터 파싱
            
            # 파싱된 데이터를 저장하고 초기화
            all_data.extend(kma_api.data)
            kma_api.data = []  # 다음 호출을 위해 초기화
            
        except Exception as e:
            print(f"오류 발생: {e}")
        
        # 다음 시간으로 이동 (1시간 단위 증가)
        current_time += timedelta(hours=1)

    # 모든 데이터를 하나의 데이터프레임으로 병합
    df = pd.DataFrame(all_data, columns=kma_api.columns)
    return df

In [ ]:
# api_key = os.getenv('weather_api_key')  # 환경 변수에서 가져오기

# # 시작 및 종료 시간 설정
# start_time = datetime(2024, 1, 1, 0, 0)   # 2024년 1월 1일 00시
# end_time = datetime(2024, 12, 31, 23, 0) # 2024년 12월 31일 23시

# # 전체 데이터 호출 및 병합
# df_all = fetch_all_data(api_key=api_key, start_time=start_time, end_time=end_time, stn="108")

# # 결과 출력
# print(df_all.head())

# # CSV 파일로 저장하기 (선택 사항)
# df_all.to_csv("kma_weather_2024.csv", index=False, sep = ',', encoding = 'utf-8')

In [ ]:
# json file 의 key 값 확인 (각 딕셔너리 값으로 이뤄져있음)
# response['rentData'].keys()

dict_keys(['list_total_count', 'RESULT', 'row'])

In [ ]:
# json 파일의 row 데이터 로드 
# response['rentData']['row']

[{'BIKE_ID': 'SPB-82123',
  'RENT_DT': '2024-12-01 01:01:12',
  'RENT_ID': '04340',
  'RENT_NM': '방배래미안아트힐아파트 앞',
  'RENT_HOLD': '99',
  'RTN_DT': '2024-12-01 01:01:15',
  'RTN_ID': '04340',
  'RTN_NM': '방배래미안아트힐아파트 앞',
  'RTN_HOLD': '99',
  'USE_MIN': '0',
  'USE_DST': '0.00',
  'USR_CLS_CD': 'USR_001',
  'BIRTH_YEAR': '1967',
  'RENT_STATION_ID': 'ST-3220',
  'RETURN_STATION_ID': 'ST-3220',
  'BIKE_SE_CD': '새싹자전거',
  'START_INDEX': 0,
  'END_INDEX': 0,
  'RNUM': '1'},
 {'BIKE_ID': 'SPB-48788',
  'RENT_DT': '2024-12-01 01:01:11',
  'RENT_ID': '02531',
  'RENT_NM': '멤피스아파트 앞',
  'RENT_HOLD': '0',
  'RTN_DT': '2024-12-01 01:01:16',
  'RTN_ID': '02531',
  'RTN_NM': '멤피스아파트 앞',
  'RTN_HOLD': '0',
  'USE_MIN': '0',
  'USE_DST': '0.00',
  'USR_CLS_CD': 'USR_001',
  'SEX_CD': 'M',
  'BIRTH_YEAR': '1968',
  'RENT_STATION_ID': 'ST-1916',
  'RETURN_STATION_ID': 'ST-1916',
  'BIKE_SE_CD': '일반자전거',
  'START_INDEX': 0,
  'END_INDEX': 0,
  'RNUM': '2'},
 {'BIKE_ID': 'SPB-38818',
  'RENT_DT': '2024-

In [ ]:
import os
import requests
import pandas as pd

class SeoulBikeHotPlaces:
    def __init__(self, api_key=None):
        """
        Google Places API 키를 초기화합니다.
        :param api_key: Google Places API 키 (환경 변수 또는 직접 입력)
        """
        self.api_key = api_key or os.getenv('gmaps_api_key')
        if not self.api_key:
            raise ValueError("Google Places API Key가 필요합니다. 환경 변수로 설정하거나 직접 전달하세요.")
    
    def get_hot_places(self, latitude, longitude, radius=500, place_type="tourist_attraction"):
        """
        특정 위치 주변의 핫플레이스를 검색합니다.
        :param latitude: 검색 중심점의 위도
        :param longitude: 검색 중심점의 경도
        :param radius: 검색 반경 (미터 단위, 기본값: 500m)
        :param place_type: 검색할 장소 유형 (예: restaurant, cafe, park 등)
        :return: 핫플레이스 정보 리스트
        """
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "location": f"{latitude},{longitude}",
            "radius": radius,
            "type": place_type,
            "key": self.api_key
        }
        
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()  # HTTP 에러 발생 시 예외 처리
            data = response.json()
            
            # 결과에서 필요한 정보 추출
            places = []
            for result in data.get("results", []):
                places.append({
                    "name": result.get("name"),
                    "latitude": result["geometry"]["location"]["lat"],
                    "longitude": result["geometry"]["location"]["lng"],
                    "rating": result.get("rating"),
                    "user_ratings_total": result.get("user_ratings_total"),
                    "address": result.get("vicinity")
                })
            
            return places
        
        except requests.exceptions.RequestException as e:
            print(f"데이터를 가져오는 중 오류가 발생했습니다: {e}")
            return []
    
    def get_places_for_stations(self, bike_stations_df, radius=500, place_type="tourist_attraction"):
        """
        자전거 대여소 데이터프레임을 기반으로 각 대여소 주변 핫플레이스를 검색합니다.
        :param bike_stations_df: 자전거 대여소 정보가 포함된 DataFrame (latitude와 longitude 열 포함)
        :param radius: 검색 반경 (미터 단위, 기본값: 500m)
        :param place_type: 검색할 장소 유형 (예: restaurant, cafe, park 등)
        :return: 대여소별 핫플레이스 정보를 포함한 DataFrame
        """
        all_places = []
        
        for _, row in bike_stations_df.iterrows():
            station_id = row["stationId"]
            station_name = row["stationName"]
            latitude = row["stationLatitude"]
            longitude = row["stationLongitude"]
            
            print(f"대여소 '{station_name}' 주변 핫플레이스 검색 중...")
            
            # 특정 대여소 주변 핫플레이스 검색
            places = self.get_hot_places(latitude, longitude, radius=radius, place_type=place_type)
            
            # 각 장소에 대여소 ID와 이름 추가
            for place in places:
                place["stationId"] = station_id
                place["stationName"] = station_name
            
            all_places.extend(places)
        
        # 결과를 DataFrame으로 변환
        return pd.DataFrame(all_places)

